# Класс Vocabulary

Первая фаза преобразования текста в векторизованный мини-пакет — отображение токенов в числовую форму. Стандартный его вариант — взаимно однозначное, то есть обратимое отображение — между токенами и числами. На языке Python они будут представлять собой два словаря. Мы инкапсулируем это взаимно однозначное соответствие в классе Vocabulary.


Класс Vocabulary не только отвечает за это взаимно однозначное соответствие, благодаря которому пользователь может добавлять новые токены и автоматически наращивать значение индекса, но и поддерживает специальный токен UNK, название которого расшифровывается как «неизвестный» (unknown). Благодаря UNK можно при контроле обрабатывать токены, которые алгоритм не видел при обучении (например, при контроле могут встретиться слова, которых не было в обучающем наборе данных). Как вы увидите в следующем пункте, мы даже будем явным образом исключать нечасто встречающиеся токены из Vocabulary, так что в процедуре обучения будут встречаться токены UNK. Они играют важную роль в уменьшении объема используемой классом Vocabulary памяти. Ожидается, что для добавления новых токенов в Vocabulary будет вызываться метод add_token(), для извлечения индекса токена — метод lookup_token() и для извлечения соответствующего конкретному индексу токена — lookup_index().

In [1]:
from argparse import Namespace
from collections import Counter
import json
import os
import re
import string

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook

In [2]:
class Vocabulary(object):
    """Класс для обработки текста и извлечения словарного запаса"""
    def __init__(self, token_to_idx=None, add_unk=True, unk_token='<UNK>'):
        """
        Аргументы:
        token_to_idx(dict): готовый ассоциативный массив соответствий токенов индексам 
        add_unk (bool): флаг, указывающий, нужно ли добавлять токен UNK
        unk_token (str): добавляемый в словарь токен UNK
        """

        if token_to_idx is None:
            token_to_idx = {}
            
        self._token_to_idx = token_to_idx
        self._idx_to_token = {idx: token for token, idx in self._token_to_idx.items()}
        self._add_unk = add_unk
        self._unk_token = unk_token
        self.unk_index = 1
        
        if add_unk:
            self.unk_index = self.add_token(unk_token)
            
    def to_serializable(self):
        """Возвращает словарь с возможностью сериализации(сохранения в файл)"""
        return {'token_to_idx': self._token_to_idx,
                'add_unk': self._add_unk,
                'unk_token': self._unk_token}
    
    @classmethod
    def from_serializable(cls, contents):
        """Создает экземпляр Vocabulary на основе сериализованного словаря"""
        return cls(**contents)
        
    def add_token(self, token):
        """ Обновляет словари отображения, добавляя в них токен.
            Аргументы:
            token (str): добавляемый в Vocabulary элемент
            Возвращает:
            index (int): соответствующее токену целочисленное значение"""
        
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def lookup_token(self, token):
        """
        Извлекает соответствующий токену индекс
        или индекс UNK, если токен не найден.
        Аргументы:
        token (str): токен для поиска
        Возвращает:
        index (int): соответствующий токену индекс
        Примечания:
        'unk_index' должен быть >=0 (добавлено в Vocabulary)
        для должного функционирования UNK
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]
        
    def lookup_index(self, index):
        """ Возвращает соответствующий индексу токен
        Аргументы:
        index (int): индекс для поиска
        Возвращает:
        token (str): соответствующий индексу токен
        Генерирует:
        KeyError: если индекс не найден в Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError('the index (%d) is not in the Vocabulary' % index)
        return self._idx_to_token[index]
    
    def __str__(self):
        return '<Vocabulary(size=%d)>' % len(self)
    
    def __len__(self):
        return len(self._token_to_idx)

# Класс ReviewVectorizer

Вторая фаза преобразования текста в векторизованный мини-пакет — проход в цикле по токенам входных данных и преобразование каждого из них в цифровую форму. В результате этого должен получиться вектор. А поскольку этот вектор затем объединяется с векторами для других точек данных, возникает ограничение, гласящее, что длина создаваемых классом Vectorizer векторов должна быть одинаковой.

In [3]:
class ReviewVectorizer(object):
    """Векторизатор, приводящий длину векторов корпусов к единому размеру"""
    
    def __init__(self, review_vocab, rating_vocab):
        """Аргументы:
                    review_vocab (Vocabulary): отображает слова
                    в целочисленные значения
                    
                    rating_vocab (Vocabulary): отображает метки классов
                    в целочисленные значения"""
        self.review_vocab = review_vocab
        self.rating_vocab = rating_vocab
        
    def vectorize(self, review):
        """Создает свернутый унитарный вектор для обзора
        Аргументы:
        review (str): обзор
        Возвращает:
        one_hot (np.ndarray): свернутое унитарное представление
        """
        
        one_hot = np.zeros(len(self.review_vocab), dtype=np.float32)
        
        for token in review.split(' '):
            if token not in string.punctuation:
                one_hot[self.review_vocab.lookup_token(token)] = 1
                
        return one_hot
    
    @classmethod
    def from_dataframe(cls, review_df, cutoff=25):
        """ Создает экземпляр векторизатора на основе
        объекта DataFrame набора данных
        Аргументы:
        review_df (pandas.DataFrame): набор данных обзоров
        cutoff (int): параметр для фильтрации по частоте вхождения
        Возвращает:
        экземпляр класса ReviewVectorizer
        """
        review_vocab = Vocabulary(add_unk=True)
        rating_vocab = Vocabulary(add_unk=False)
        
        # добавляем рейтинг
        for rating in sorted(set(review_df.rating)):
            rating_vocab.add_token(rating)
            
        # Добавляем часто используемые слова, если число вхождений больше указанного
        word_counts = Counter()
        for review in review_df.review:
            for word in review.split(" "):
                if word not in string.punctuation:
                    word_counts[word] += 1
                    
        for word, count in word_counts.items():
            if count > cutoff:
                review_vocab.add_token(word)
                
        return cls(review_vocab, rating_vocab)
    
    @classmethod
    def from_serializable(cls, contents):
        """
        Создает экземпляр ReviewVectorizer на основе
        сериализуемого словаря
        Аргументы:
        contents (dict): сериализуемый словарь
        Возвращает:
        экземпляр класса ReviewVectorizer
        """
        review_vocab = Vocabulary.from_serializable(contents['review_vocab'])
        rating_vocab = Vocabulary.from_serializable(contents['rating_vocab'])
        
        return cls(review_vocab=review_vocab, rating_vocab=rating_vocab)
    
    def to_serializable(self):
        """
        Создает сериализуемый словарь для кэширования
        Возвращает:
        contents (dict): сериализуемый словарь
        """
        return {'review_vocab': self.review_vocab.to_serializable(),
                'rating_vocab': self.rating_vocab.to_serializable()}

# DataLoader(Класс ReviewDataset)

Последняя фаза конвейера преобразования текста в векторизованный мини-пакет — собственно группировка векторизованных точек данных. Поскольку группировка в мини-пакеты играет столь важную роль в обучении нейронных сетей, фреймворк PyTorch предоставляет для координации этого процесса встроенный класс DataLoader. Для создания экземпляра класса DataLoader необходимо передать какой-либо объект Dataset PyTorch (например, описанный нами для этого примера ReviewDataset), batch_size и несколько других поименованных аргументов. В результате получается объект, представляющий собой Python-итератор, группирующий и свертывающий содержащиеся в объекте Dataset точки данных/

In [4]:
class ReviewDataset(Dataset):
    def __init__(self, review_df, vectorizer):
        """
        Аргументы:
            review_df (pandas.DataFrame): датасет
            vectorizer (ReviewVectorizer): векторизованное представление датасета
        """
        self.review_df = review_df
        self._vectorizer = vectorizer
        
        self.train_df = self.review_df[self.review_df.split=='train']
        self.train_size = len(self.train_df)
        
        self.val_df = self.review_df[self.review_df.split=='val']
        self.val_size = len(self.val_df)
        
        self.test_df = self.review_df[self.review_df.split=='test']
        self.test_size = len(self.test_df)
        
        self._lookup_dict = {'train': (self.train_df, self.train_size), 
                             'val': (self.val_df, self.val_size),
                             'test': (self.test_df, self.test_size),}
        self.set_split('train')
        
    @classmethod
    def load_dataset_and_make_vectorizer(cls, review_csv):
        """
        Загрузить набор данных и создать векторизированную форму с нуля
        Аргументы:
        review_csv (str): расположение датасета
        Returns:
            экземпляр ReviewDataset
        """
        review_df = pd.read_csv(review_csv)
        train_review_df = review_df[review_df.split=='train']
        return cls(review_df, ReviewVectorizer.from_dataframe(train_review_df))
    
    @classmethod
    def load_dataset_and_load_vectorizer(cls, review_csv, vectorizer_filepath):
        '''
        Загрузка датасета и соответствующего векторизатора
        Используется в том случае, когда векторизатор был кэширован для повторного использования
        
        Аргументы:
        vectorizer_filepath (str): расположение векторизатора
        Returns:
            экземпляр ReviewVectorizer
        '''
        with open(vectorizer_filepath) as fp:
            return ReviewVectorizer.from_serializable(json.load(fp))
        
    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        '''
        Статический метод для только загрузки векторизатора из файла
        '''
        with open(vectorizer_filepath) as fp:
            return ReviewVectorizer.from_serializable(json.load(fp))
        
    def save_vectorizer(self, vectorizer_filepath):
        '''Сохраняет векторизатор в файл json'''
        with open(vectorizer_filepath, 'w') as fp:
            json.dump(self._vectorizer.to_serializable(), fp)
            
    def get_vectorizer(self):
        '''Возвратить векторизатор'''
        return self._vectorizer
    
    def set_split(self, split='train'):
        '''Выбрать столбец для разбиения датасета на подвыборки
        аргументы:
            split(str): один из 'train', 'val', 'test'
        '''
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]
        
        """для наследования класса Dataset PyTorch разработчик должен реализовать методы __getitem__() и __len__(), 
        благодаря чему класс DataLoader сможет пройти в цикле по набору данных путем итерации по индексам из этого набора."""
        
    def __len__(self):
        return self._target_size
    
    def __getitem__(self, index):
        '''Первичная точка входа для датасетов Pytorch
        args:
            index(int): индекс для доступа к значениям данным
        Return:
            словарь, содержащий значения данных (x_data) и метку (y_target)
        '''
        row = self._target_df.iloc[index]
        
        review_vector = self._vectorizer.vectorize(row.review)
        rating_index = self._vectorizer.rating_vocab.lookup_token(row.rating)
        
        return {'x_data': review_vector, 
                'y_target': rating_index}
    
    def get_num_batches(self, batch_size):
        '''Возвращает количество батчей в датасете'''
        return len(self) // batch_size

Создадим для DataLoader адаптер в виде функции generate_batches() — генератора для удобного выбора (switch) данных между CPU и GPU.

In [5]:
def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device='cpu'):
    '''
    Функция-генератор — адаптер для объекта DataLoader фреймворка PyTorch.
    Гарантирует размещение всех тензоров на нужном устройстве.
    '''
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

# Классификатор-перцептрон

Используемая в этом примере модель представляет собой новую реализацию классификатора Perceptron. Класс ReviewClassifier наследует класс Module фреймворка PyTorch и создает слой преобразования типа Linear, возвращающего один результат. Поскольку речь идет о бинарной классификации (обзор может быть позитивным или негативным), этого вполне достаточно. В качестве завершающего нелинейного преобразования используется сигма-функция.

Благодаря параметризации метода forward() применение сигма-функции необязательно. Чтобы разобраться, для чего это нужно, укажем сначала, что наиболее подходящей функцией потерь для задачи бинарной классификации является бинарная функция потерь на основе перекрестной энтропии (torch.nn.BCELoss()). Она специально сформулирована математически в расчете на бинарные вероятности. Однако в случае применения сигма-функции, а затем этой функции потерь возникают проблемы с численной устойчивостью. PyTorch предоставляет пользователям более устойчивый численно вариант — BCEWithLogitsLoss(). При использовании функции потерь не следует применять сигма-функцию (поэтому по умолчанию она у нас не применяется). Но если пользователю классификатора хотелось бы получить вероятностное значение, понадобится сигма-функция, так что такую возможность необходимо оставить в качестве необязательной.

In [6]:
class ReviewClassifier(nn.Module):
    '''Классификатор на основе перцептрона'''
    def __init__(self, num_features):
        '''
        num_features: размер входного вектора признаков
        '''
        super(ReviewClassifier, self).__init__()
        self.fc1 = nn.Linear(in_features=num_features, out_features=1)
        
    def forward(self, x_in, apply_sigmoid=False):
        '''Прямой проход классификатора
        Аргументы:
            x_in (torch.Tensor): входной тензор данных
            x_in.shape должен быть (batch, num_features)
            apply_sigmoid (bool): флаг для сигма-функции активации
            при использовании функции потерь на основе перекрестной
            энтропии должен равняться false
        Возращает:
            итоговый тензор. tensor.shape должен быть (batch,).
        '''
        y_out = self.fc1(x_in).squeeze()
        if apply_sigmoid:
            y_out = F.sigmoid(y_out)
        return y_out

# Процедура обучения

В этом подразделе мы в общих чертах опишем компоненты процедуры обучения и их взаимодействие с набором данных и моделью с целью подбора параметров модели и повышения ее эффективности. По сути, процедура обучения отвечает за воплощение модели, проход в цикле по набору данных, вычисление (на основе переданных входных данных) выходных значений модели, определение потерь (того, насколько модель ошиблась в предсказаниях) и обновление модели в соответствии с потерями. Хотя на первый взгляд здесь есть масса нюансов, на самом деле точек модификации процедуры обучения не так уж много, поэтому вы скоро привыкнете к ней в процессе разработки алгоритмов глубокого обучения.

Для упрощения управления высокоуровневыми решениями мы сосредоточим в объекте args все точки принятия решений.

In [24]:
args = Namespace(
    # информация о данных и путях
    frequency_cutoff=25, 
    model_state_file='model.pth', 
    review_csv='data/yelp/reviews_with_splits_full.csv',
    save_dir='model_storage/ch3/yelp',
    vectorizer_file='vectorizer.json',
    # нет гиперпараметров модели
    # есть гиперпараметры обучения
    batch_size=128,
    early_stopping_criteria=5,
    learning_rate=0.001,
    num_epochs=5,
    seed=1137,
    # параметры запуска
    catch_keyboard_interrupt=True,
    cuda=True,
    expand_filepaths_to_save_dir=True,
    reload_from_files=False,
)

## Вспомогательные функции

**Зададим состояние обучения**

In [25]:
def make_train_state(args):
    
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

Обработка обновлений состояний обучения

In [39]:
def update_train_state(args, model, train_state):
    """
    Составные части:
      - Ранняя остановка: предотвратить переобучение.
      - Контрольная точка модели: модель сохраняется, если модель лучше
    Аргументы:
     - param args: основные аргументы
     - парам модель: модель к обучению
     - param train_state: словарь, представляющий значения состояния обучения
     - возвращает:
         новый train_state
    """
    
    # сохранить хотябы одну модель
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False
        
    # сохранить, если модель улучшилась
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]
        
        # если Loss ухудшился(увеличился)
        if loss_t >= train_state['early_stopping_best_val']:
            # обновить шаг
            train_state['early_stopping_step'] += 1
        # loss уменьшился
        else:
            # нухно сохранить лучшую модель
            torch.save(model.state_dict(), train_state['model_filename'])
            # сбросим шаг ранней остановки
            train_state['early_stopping_step'] = 0
            
        # остановить ранее?
        train_state['stop_early'] = train_state['early_stopping_step'] >= args.early_stopping_criteria
    
    return train_state

Расчет accuracy:

In [27]:
def compute_accuracy(y_pred, y_target):
    y_target = y_target.cpu()
    y_pred_indices = (torch.sigmoid(y_pred)>0.5).cpu().long()
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

Еще полезные функции:

In [28]:
# инициализация генератора случайных чисел
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)
        
# проверка на существование директорий
def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

In [29]:
# Добавим пути к файлам
if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = args.save_dir + '/' + args.vectorizer_file
    args.model_state_file = args.save_dir + '/' + args.model_state_file
    print('Расширенные пути к файлам: ')
    print('\t{}'.format(args.vectorizer_file))
    print('\t{}'.format(args.model_state_file))

Расширенные пути к файлам: 
	model_storage/ch3/yelp/vectorizer.json
	model_storage/ch3/yelp/model.pth


In [30]:
# проверим доступность CUDA
if not torch.cuda.is_available():
    args.cuda = False
    print('Not')

In [31]:
print("Использование CUDA: {}".format(args.cuda))

Использование CUDA: True


In [32]:
args.device = torch.device("cuda" if args.cuda else "cpu")

In [33]:
# зададим параметры воспроизводимости результатов
set_seed_everywhere(args.seed, args.cuda)

In [34]:
# проверим и создадим директорию для сохранения файлов
handle_dirs(args.save_dir)

**Готовим датасет**

In [35]:
if args.reload_from_files:
    # из сохраненного
    print('Подгружаем датасет и векторизатор')
    dataset = ReviewDataset.load_dataset_and_load_vectorizer(args.review_csv, args.vectorizer_file)
else:
    print('Подгружаем датасет и создаем векторизатор')
    dataset = ReviewDataset.load_dataset_and_make_vectorizer(args.review_csv)
    dataset.save_vectorizer(args.vectorizer_file)

vectorizer = dataset.get_vectorizer()

# создадим объект классификатора
classifier = ReviewClassifier(num_features=len(vectorizer.review_vocab))    

Подгружаем датасет и создаем векторизатор


In [36]:
classifier = classifier.to(args.device) # зададим параметры классификатора в зависимости от способа обучения - на CPU или на GPU

In [40]:
# выберем Loss функцию BCE с логистическими потерями
loss_func = nn.BCEWithLogitsLoss()

# воспользуемся оптимизатором Adam
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)

# определим планировщик:
# позволяет снизить скорость динамического обучения на основе некоторых проверочных измерений.
sheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.5, patience=1)

# зададим состояние обучения
train_state = make_train_state(args)

# отображение эпох
epoch_bar = tqdm_notebook(desc='training routine', 
                          total=args.num_epochs, 
                          position=0)

# для обучающей части
dataset.set_split('train')

# отображение обучающей части
train_bar = tqdm_notebook(desc='split=train', 
                          total=dataset.get_num_batches(args.batch_size), 
                          position=1, 
                          leave=True)

# для валидационной части
dataset.set_split('val')
val_bar = tqdm_notebook(desc='split=val', 
                        total=dataset.get_num_batches(args.batch_size), 
                        position=1, 
                        leave=True)

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index
        """
         # Итерации по набору обучающих данных

         # настройка: пакетный генератор, установить потери и метрику на 0, включить режим обучения
        """
        dataset.set_split('train')
        batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
        
        running_loss = 0.0
        running_acc = 0.0
        
        # указываем, что параметры модели могут изменяться
        classifier.train()
        
        for batch_index, batch_dict in enumerate(batch_generator):
            # Тренеровочная программа состоит из 5 шагов
            
            # Шаг 1. Обнуление градиентов
            optimizer.zero_grad()
            
            # Шаг 2. Вычесляем вывод (предсказание)
            y_pred = classifier(x_in=batch_dict['x_data'].float())
            
            # Шаг 3. Вычисляем потери
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            
            # Шаг 4. Используем потери для создания градиентов
            loss.backward()
            
            # Шаг 5. Используем оптимизатор, чтобы сделать шаг градиента
            optimizer.step()
            
            
            # ------------------------------------------------
            # Вычисляем accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            
            # обновить столбик отображения
            train_bar.set_postfix(loss=running_loss, acc=running_acc, epoch=epoch_index)
            train_bar.update()
        
        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)
        
        # Проход в цикле по проверочному набору данных
        # настройка: создаем генератор пакетов, устанавливаем значения
        # переменных loss и acc равными 0, включаем режим проверки
        
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
        running_loss = 0.
        running_acc = 0.
        
        # указываем, что параметры модели неизменяемы и отключаем дропаут
        classifier.eval()
        
        for batch_index, batch_dict in enumerate(batch_generator):
            
            # делаем предсказание
            y_pred = classifier(x_in=batch_dict['x_data'].float())
            
            # шаг 3. вычисляем потери
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)
            
            # вычисляем accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            
            val_bar.set_postfix(loss=running_loss, acc=running_acc, epoch=epoch_index)
            val_bar.update()
            
        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)
        
        train_state = update_train_state(args=args, model=classifier, train_state=train_state)
        sheduler.step(train_state['val_loss'][-1])
        
        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
        
        if train_state['stop_early']:
            break
            
        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()

except KeyboardInterrupt:
    print("Exiting loop")

c:\users\rusancovs\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


c:\users\rusancovs\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


c:\users\rusancovs\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Exiting loop


Цикл обучения использует созданные в начале объекты для обновления параметров модели так, чтобы ее эффективность со временем росла. Точнее говоря, цикл обучения состоит из двух циклов: внутреннего цикла по мини-пакетам из набора данных и внешнего цикла, повторяющего внутренний нужное число раз. Во внутреннем цикле для каждого мини-пакета вычисляется функция потерь, а параметры модели обновляются с помощью оптимизатора.

Код оценки эффективности модели на выделенном контрольном наборе почти ничем не отличается от цикла проверки в процедуре обучения из предыдущего примера с одним лишь небольшим нюансом: задан фрагмент 'test', а не 'val'. Различие между этими двумя фрагментами набора данных состоит в том, что контрольный набор следует использовать как можно меньше. При каждом запуске обученной модели на контрольном наборе, принятии на основании этого новых решений относительно модели (например, изменения размера слоев) и повторной оценки заново обученной модели на контрольном наборе модельные решения смещаются в сторону контрольных данных. Другими словами, при частом повторении этого процесса контрольные данные не смогут служить точной мерой, как действительно выделенные данные.

In [43]:
# используем лучшие параметры Loss и accuracy для проверок на тестовой выборки.

classifier.load_state_dict(torch.load(train_state['model_filename']))
classifier = classifier.to(args.device)

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred = classifier(x_in=batch_dict['x_data'].float())

    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [44]:
print("Потери на тесте: {:.3f}".format(train_state['test_loss']))
print("Accuracy на тесте: {:.2f}".format(train_state['test_acc']))

Потери на тесте: 0.178
Accuracy на тесте: 93.56


# Вывод

Еще один метод оценки эффективности модели — вывод на основе новых данных и анализ того, работает ли она. Добавим обработчик текста:

In [45]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

In [46]:
def predict_rating(review, classifier, vectorizer, decision_threshold=0.5):
    """
    Предсказание рейтинга обзора
    Аргументы:
        review (str): текст обзора
        classifier (ReviewClassifier): обученная модель
        vectorizer (ReviewVectorizer): соответствующий векторизатор
        decision_threshold (float): численная граница,
        разделяющая различные классы рейтинга
    
    """
    
    review = preprocess_text(review)
    
    vectorized_review = torch.tensor(vectorizer.vectorize(review))
    result = classifier(vectorized_review.view(1, -1))
    
    probability_value = F.sigmoid(result).item()
    index = 1
    if probability_value < decision_threshold:
        index=0
    
    return vectorizer.rating_vocab.lookup_index(index)

In [47]:
test_review = 'this is a pretty awesome book'

classifier = classifier.cpu()
prediction = predict_rating(test_review, classifier, vectorizer, decision_threshold=0.5)
print(f"{test_review} - > {prediction}")

this is a pretty awesome book - > 2


c:\users\rusancovs\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Просмотр весов модели

Наконец, последний способ выяснить, хорошо ли работает модель после окончания обучения, — просмотреть веса модели и принять решение, правильно ли они выглядят. В случае перцептрона и свернутого унитарного кодирования эта задача достаточно проста, поскольку каждому весу модели соответствует ровно одно слово из словаря.

In [50]:
# сортируем веса
fc1_weights = classifier.fc1.weight.detach()[0]
_, indices = torch.sort(fc1_weights, dim=0, descending=True)
indices = indices.numpy().tolist()

# Топ 20 слов
print("Наиболее часто встречающиеся слова в позитивных откликах:")
print("--------------------------------------")
for i in range(20):
    print(vectorizer.review_vocab.lookup_index(indices[i]))
    
print("====\n\n\n")

# Топ 20 негативных слов
print("Наиболее часто встречающиеся слова в негативных откликах:")
print("--------------------------------------")
indices.reverse()
for i in range(20):
    print(vectorizer.review_vocab.lookup_index(indices[i]))

Наиболее часто встречающиеся слова в позитивных откликах:
--------------------------------------
exceeded
pleasantly
delicious
excellent
hooked
incredible
hesitate
addicted
deliciousness
downside
fantastic
perfection
disappoint
amazing
yummmm
nexcellent
divine
delicioso
yum
awesome
====



Наиболее часто встречающиеся слова в негативных откликах:
--------------------------------------
poisoning
mediocre
worst
slowest
meh
underwhelmed
overrated
tasteless
rudest
downhill
bland
horrible
redeeming
nwon
flavorless
inedible
unacceptable
terrible
disappointing
unimpressed
